# Propagation on path graph

## Loading modules

In [ ]:
using QSWalk
using PyPlot # for plot
using LightGraphs # for PathGraph

## Evolution on path graph for fixed time, global vs local interaction

Basic parameters. Note *dim* should be odd number. Otherwise middle point *midpoint* is not unique. 
*adjacency* can be generated as *adjacency = spdiagm((ones(dim-1), ones(dim-1)), (-1, 1))*.

In [ ]:
dim = 251 #odd for unique middle point
midpoint = ceil(Int, dim/2)
w = 0.5
timepoint = 100.
adjacency = adjacency_matrix(PathGraph(dim));

Quantum stochastic walk: global interaction case. Note *evolve_generator* requires list of Lindblad operators, hence we need extra brackets

In [ ]:
opglobal = evolve_generator(adjacency, [adjacency], w)
rhoglobal = evolve(opglobal, proj(midpoint, dim), timepoint);

Quantum stochastic walk: local interaction case. Note *lindlocal* is already a list of Lindbladian operators, hence we do not need extra brackets contrary to global interaction case.

In [ ]:
lindlocal = classical_lindblad_operators(adjacency)
oplocal = evolve_generator(adjacency, lindlocal, w)
rholocal = evolve(oplocal, proj(midpoint, dim), timepoint);

To plot the result of cannonical measurement, we simply take a diagonal of the states. Note that both *rhoglobal* and *rholocal* are complex matrices, hence we need to take real part. *positions* is generated assuming *midpoint* (initial position) is at 0. Note that we have very heavy tails in global interaction case which confirms fasr propagation of the model.

In [ ]:
positions = (collect(1:dim) - midpoint)
plot(positions, real.(diag(rholocal)), "k")
plot(positions, real.(diag(rhoglobal)), "b")
xlabel("position")
ylabel("probability")
show()

## Evolution on path graph for fixed time, propagation analysis

Basic parameters. Note *dim* should be odd number. Otherwise middle point *midpoint* is not unique. 
*adjacency* can be generated as *adjacency = spdiagm((ones(dim-1), ones(dim-1)), (-1, 1))*.

In [ ]:
dim = 251 #odd for unique middle point
midpoint = ceil(Int, dim/2)
w = 0.5
timepoints = collect(0:5:100)
adjacency = adjacency_matrix(PathGraph(dim));

Preparation of global and local interaction operators.

In [ ]:
opglobal = evolve_generator(adjacency, [adjacency], w)
lindlocal = classical_lindblad_operators(adjacency)
oplocal = evolve_generator(adjacency, lindlocal, w);

Calculating data. Note that comparing to global operators, density states usually are small matrices. Hence we can store those. Furthermore note, that evolve allows to compute the same evolutions at different points of time.

In [ ]:
globalstates = evolve(opglobal, proj(midpoint, dim), timepoints)
localstates = evolve(oplocal, proj(midpoint, dim), timepoints);

Second moment data calculation

In [ ]:
secmomentglobal = Float64[]
secmomentlocal = Float64[]
positions = (collect(1:dim) - midpoint)

for (rhoglobal, rholocal) = zip(globalstates, localstates)
  push!(secmomentglobal, sum(positions.^2 .* diag(rhoglobal)))
  push!(secmomentlocal, sum(positions.^2 .* diag(rholocal)))
end

Plots of the results. Note that second moment for global interaction case grows ballisticaly.

In [ ]:
plot(timepoints, secmomentlocal, "k")
plot(timepoints, secmomentglobal, "b")
xlabel("t")
ylabel(L"\mu_2")
show()